In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv(r'C:\Users\user\Documents\Data Science\bestsellers with categories.csv')
df.head()

,Name,Author,User Rating,Reviews,Price,Year,Genre
0,10-Day Green Smoothie Cleanse,JJ Smith,4.7,17350,8,2016,Non Fiction
1,11/22/63: A Novel,Stephen King,4.6,2052,22,2011,Fiction
2,12 Rules for Life: An Antidote to Chaos,Jordan B. Peterson,4.7,18979,15,2018,Non Fiction
3,1984 (Signet Classics),George Orwell,4.7,21424,6,2017,Fiction
4,"5,000 Awesome Facts (About Everything!) (Natio...",National Geographic Kids,4.8,7665,12,2019,Non Fiction


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 550 entries, 0 to 549
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   Name         550 non-null    object 
 1   Author       550 non-null    object 
 2   User Rating  550 non-null    float64
 3   Reviews      550 non-null    int64  
 4   Price        550 non-null    int64  
 5   Year         550 non-null    int64  
 6   Genre        550 non-null    object 
dtypes: float64(1), int64(3), object(3)
memory usage: 30.2+ KB


____
# 1. One Sample T-Test
**Hypothesis:**  
**Ho:** Mean price = 10  
**Ha:** Mean price > 10

In [6]:
from scipy.stats import ttest_1samp
tset, pval = ttest_1samp(df['Price'], 10)
print('p-value: ', pval/2)

# Conclusion
if pval/2 < 0.05:
    print('Reject Ho')
else:
    print('Accept Ho')

p-value:  2.5008144800630206e-11
Reject Ho


In [7]:
df['Price'].mean()

13.1

_____
# 2. Two Sample Independent T-Test  
**Hypothesis:**  
**Ho:** mean fiksi = mean non-fiksi  
**Ha:** mean fiksi > mean non-fiksi


In [8]:
df[['Price', 'Genre']].head()

,Price,Genre
0,8,Non Fiction
1,22,Fiction
2,15,Non Fiction
3,6,Fiction
4,12,Non Fiction


## a) Non Equal Variance

In [11]:
from scipy.stats import ttest_ind

result = list(ttest_ind
              (df[df['Genre'] == 'Non Fiction']['Price'],
               df[df['Genre']== 'Fiction']['Price'],
              equal_var=False))

print('T-Statistic: ', result[0])
print('p-value: ', result[1]/2)

if result[1]/2 < 0.05:
    print('Reject Ho')
else:
    print('Accept Ho')

T-Statistic:  4.537208093329263
p-value:  3.5095048008863536e-06
Reject Ho


## b) Equal Variance

In [12]:
from scipy.stats import ttest_ind

result = list(ttest_ind
              (df[df['Genre'] == 'Non Fiction']['Price'],
               df[df['Genre']== 'Fiction']['Price'],
              equal_var=True))

print('T-Statistic: ', result[0])
print('p-value: ', result[1]/2)

if result[1]/2 < 0.05:
    print('Reject Ho')
else:
    print('Accept Ho')

T-Statistic:  4.351608076818746
p-value:  8.062175359060365e-06
Reject Ho


In [13]:
df[['Price', 'Genre']].groupby(['Genre']).mean()

,Price
Genre,
Fiction,10.850000
Non Fiction,14.841935


## c) T-Test Paired

In [14]:
using_smartphone = [604, 556, 540, 522, 469, 544, 513, 470, 556, 531, 599, 537, 619, 536, 554, 467]
no_smartphone = [636, 623, 615, 672, 601, 600, 542, 554, 543, 520, 609, 559, 595, 565, 573, 554]

df_paired = pd.DataFrame({
    'No':no_smartphone,
    'Yes':using_smartphone
})

df_paired['diff'] = df_paired['Yes'] - df_paired['No']

In [15]:
df_paired.head()

,No,Yes,diff
0,636,604,-32
1,623,556,-67
2,615,540,-75
3,672,522,-150
4,601,469,-132


**Ho: Mean population of diff = 0  
Ha: Mean population of diff < 0**

In [17]:
tset, pval = ttest_1samp(df_paired['diff'], 0)
print('tset: ', tset)
print('p-value: ', pval/2)

if pval/2 < 0.05:
    print('Reject Ho')
else:
    print('Accept Ho')

tset:  -3.7091357792693707
p-value:  0.0010497189034628326
Reject Ho


## d) One Way Anova F-Test for More than Two Population

In [22]:
transport = []
method = ['Walking ', 'Public ', 'Private ']
for i in method:
    transport.extend((i*10).split())
    
df = pd.concat([pd.Series(transport, name = 'Transportation'),
               pd.Series([5,4,5,6,5,3,2,4,5,5,7,7,5,6,4,6,5,5,6,7,5,4,5,6,6,5,5,5,7,7], name = 'Study Hour')], axis=1)
df.sample(5)

,Transportation,Study Hour
22,Private,5
1,Walking,4
12,Public,5
28,Private,7
18,Public,6


**Ho: Mean study hour walking =  Mean study hour public =  Mean study hour private   
Ha: Minimal ada satu pasang yang sama**

In [25]:
from scipy.stats import f_oneway

stat, pval = f_oneway(df[df['Transportation'] == 'Public']['Study Hour'],
                      df[df['Transportation'] == 'Walking']['Study Hour'],
                      df[df['Transportation'] == 'Private']['Study Hour'])

print('Statistic: ', stat)
print('P-Value: ', pval)

if pval < 0.05:
    print('Reject Ho')
else:
    print('Accept Ho')

Statistic:  4.809836065573772
P-Value:  0.0163405142361861
Reject Ho


In [26]:
df.groupby('Transportation').mean()

,Study Hour
Transportation,
Private,5.5
Public,5.8
Walking,4.4
